# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [58]:
# Import libraries
import numpy as np
import pandas as pd

In [59]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [60]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns)-1
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = len(student_data[student_data['passed'] == 'no'])
grad_rate = float(n_passed) / float(n_students) * 100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [61]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [62]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [63]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

### Baseline Predictor Always Say "No Intervention"
#### Accuracy and F1 for Training and Test Set

In [64]:
n_passed_train = len(y_train[y_train == 'yes'])
n_failed_train = len(y_train[y_train == 'no'])
accuracy_train = float(n_passed_train) / float(len(y_train))
f1_train = 2 * 1 * accuracy_train / (1+accuracy_train)
print 'Accuracy Training Set {:.3}'.format(accuracy_train)
print 'F1 Training Set {:.3}'.format(f1_train)

n_passed_test = len(y_test[y_test == 'yes'])
n_failed_test = len(y_test[y_test == 'no'])
accuracy_test = float(n_passed_test) / float(len(y_test))
f1_test = 2 * 1 * accuracy_test / (1+accuracy_test)
print 'Accuracy Test Set {:.3}'.format(accuracy_test)
print 'F1 Test Set {:.3}'.format(f1_test)

accuracy_all = float(n_passed) / float(len(y_all))
f1_all = 2 * 1 * accuracy_all / (1+accuracy_all)
print 'Accuracy Full Set {:.3}'.format(accuracy_all)
print 'F1 Full Set {:.3}'.format(f1_all)



Accuracy Training Set 0.683
F1 Training Set 0.812
Accuracy Test Set 0.632
F1 Test Set 0.774
Accuracy Full Set 0.671
F1 Full Set 0.803


In [65]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training LogisticRegression...
Done!
Training time (secs): 0.007


In [66]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.846846846847


In [67]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.805970149254


In [68]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
sample_size = [100,200,300]
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
for size in sample_size:
    train_predict(clf, X_train[:size],y_train[:size],X_test,y_test)


------------------------------------------
Training set size: 100
Training LogisticRegression...
Done!
Training time (secs): 0.003
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.859259259259
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.764705882353
------------------------------------------
Training set size: 200
Training LogisticRegression...
Done!
Training time (secs): 0.003
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.85618729097
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.791366906475
------------------------------------------
Training set size: 300
Training LogisticRegression...
Done!
Training time (secs): 0.005
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.846846846

### Model 2 - SVM

In [69]:
# TODO: Train and predict using two other models
sample_size = [100,200,300]
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = SVC()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
#C_range = np.logspace(-2, 10, 13)
#gamma_range = np.logspace(-9, 3, 13)
for size in sample_size:
    train_predict(clf, X_train_scaled[:size],y_train[:size],X_test_scaled,y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.927536231884
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.773333333333
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.896103896104
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.765100671141
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.011
Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for training set: 0.903083700441
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.77027027027


### Model 3 - Random Forest

In [70]:
# TODO: Train and predict using two other models
sample_size = [100,200,300]
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
clf = RandomForestClassifier()
for size in sample_size:
    train_predict(clf, X_train[:size],y_train[:size],X_test,y_test)

------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.039
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.992248062016
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 200
Training RandomForestClassifier...
Done!
Training time (secs): 0.038
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 1.0
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.793893129771
------------------------------------------
Training set size: 300
Training RandomForestClassifier...
Done!
Training time (secs): 0.035
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for tra

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [71]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV

# log space values ofr C and gamma
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)

# scale the train and test set
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#make f1-scorer
from sklearn.metrics import make_scorer
scorer = make_scorer(f1_score, pos_label='yes')

# run GridSearch
clf = GridSearchCV(SVC(), param_grid=param_grid, scoring=scorer)
clf.fit(X_train_scaled, y_train)
train_f1_score = predict_labels(clf, X_train_scaled, y_train)
print "F1 score for training set: {}".format(train_f1_score)

test_f1_score = predict_labels(clf, X_test_scaled, y_test)
print "F1 score for test set: {}".format(test_f1_score)

print("The best parameters are %s with a score of %0.2f"
      % (clf.best_params_, clf.best_score_))

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.852390852391
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.76
The best parameters are {'C': 1.0, 'gamma': 0.01} with a score of 0.83


In [72]:


clf = SVC(C=1, gamma=0.01)
clf.fit(X_train_scaled, y_train)
train_f1_score = predict_labels(clf, X_train_scaled, y_train)
print "F1 score for training set: {}".format(train_f1_score)

test_f1_score = predict_labels(clf, X_test_scaled, y_test)
print "F1 score for test set: {}".format(test_f1_score)

print clf.n_support_

Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.852390852391
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.76
[ 94 134]


#### As the number of support vectors is pretty high. I also tried to check the performance of Logistic regression and see how it performs

In [73]:
from sklearn.grid_search import GridSearchCV

param_grid = dict(C=C_range)
grid= GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring=scorer)
grid.fit(X_train, y_train)

train_f1_score = predict_labels(grid, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

test_f1_score = predict_labels(grid, X_test, y_test)
print "F1 score for test set: {}".format(test_f1_score)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))


Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.830434782609
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.785714285714
The best parameters are {'C': 0.10000000000000001} with a score of 0.82


In [74]:
clf = LogisticRegression(C=0.1)

clf.fit(X_train, y_train)
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

test_f1_score = predict_labels(clf, X_test, y_test)
print "F1 score for test set: {}".format(test_f1_score)

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.830434782609
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.785714285714


In [75]:
weights = zip(list(X_train.columns), clf.coef_[0]) 

sorted(weights,key=lambda x: abs(x[1]), reverse=True)


[('failures', -0.59130627491389531),
 ('goout', -0.3989501198433168),
 ('famsup', -0.38997380932758507),
 ('higher', 0.34245460936305355),
 ('paid', 0.31118128955490021),
 ('Mjob_teacher', -0.23224414924118827),
 ('Mjob_services', 0.21571839252286457),
 ('reason_reputation', 0.20534331716607968),
 ('famrel', 0.20050633356820671),
 ('Mjob_health', 0.19721806684035928),
 ('schoolsup', -0.19072625632732496),
 ('sex_M', 0.17911188738168823),
 ('Fjob_other', 0.17119575898688302),
 ('studytime', 0.16226147610302788),
 ('Medu', 0.16027189196105737),
 ('reason_course', -0.15624817700113014),
 ('Pstatus_A', 0.14862098482785605),
 ('freetime', 0.14418645074337824),
 ('guardian_mother', 0.14340419495140372),
 ('internet', 0.13767255060153738),
 ('Walc', 0.11745164668670141),
 ('nursery', -0.10950594020925147),
 ('famsize_LE3', 0.10457463234948319),
 ('Mjob_other', -0.097867967463939234),
 ('sex_F', -0.088574473324503925),
 ('romantic', -0.086439027161888032),
 ('reason_other', 0.06978788016001509

In [76]:
pd.set_option('display.max_rows', 200)
print student_data[student_data['passed'] == 'no'][['failures','goout','famsup','higher','paid']]

     failures  goout famsup higher paid
0           0      4     no    yes   no
1           0      3    yes    yes   no
7           0      4    yes    yes   no
10          0      3    yes    yes  yes
18          3      5    yes    yes   no
24          0      2    yes    yes  yes
25          2      2    yes    yes  yes
35          0      1    yes    yes   no
44          1      3     no    yes   no
45          0      2    yes    yes  yes
49          1      4    yes    yes   no
58          0      2    yes    yes   no
62          0      3     no    yes   no
63          0      4    yes    yes  yes
67          0      3    yes    yes  yes
68          0      3    yes    yes  yes
72          2      4    yes    yes   no
79          0      3    yes    yes   no
82          0      4    yes    yes  yes
85          2      4     no    yes  yes
86          0      4    yes    yes   no
89          0      3    yes    yes   no
90          0      3    yes    yes  yes
92          0      3    yes    yes   no
